In [1]:
import numpy as np
from scipy.special import erf
import os
import sys
import time
import copy
import logging
import matplotlib.pyplot as plt

# Navigate to the parent directory of the project structure
project_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
src_dir = os.path.join(project_dir, 'src')
log_dir = os.path.join(project_dir, 'log')
fig_dir = os.path.join(project_dir, 'fig')

# Add the src directory to sys.path
sys.path.append(src_dir)


from utils.constants import WORKERS, BATCH_SAMPLES
from utils.commons import convert_bytes_to_mb, convert_bytes_to_gb
from Dataset.flight_dataset_for_LS import load_flight_dataset_matrices_for_LS, save_flight_dataset_matrices_for_LS
from Dataset.cancer_dataset_for_LS import main_generate_cancer_matrices_for_LS, load_cancer_dataset_matrices_for_LS
from Dataset.songs_dataset_for_LS import load_song_dataset_matrices_for_LS
from analysis.commons import data_normalize_by_features, concatenate_B_b, twoNorm, data_normalize_by_sample

from RP_mechanisms.optim_RP import parallel_gen_samples
from RP_mechanisms.optim_RP_DP import compute_largest_l2
from LS_mechanisms.optim_LS import lev_evaluate_ALS
from LS_mechanisms.alt19_LS import ALT19LS_mech
from LS_mechanisms.optim_LS_DP import ALS, ALS_mech, LS_fromoptim_RP_mech, OptimalLS_mech
from LS_mechanisms.LS_accuracy_estimator import LS_accuracy_estimator

from analysis.commons import compute_xopt, split_to_B_b

In [2]:
logfile_path = os.path.join(log_dir, 'LS-cancer-default.log')
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(message)s",
    handlers=[
        logging.FileHandler(logfile_path),
        logging.StreamHandler()
    ]
)

def store_array_str(data_array):
    converted_str = np.array_str(data_array)
    converted_str = converted_str.replace(' ',',')
    return converted_str

file_X_path = os.path.join(project_dir, "Dataset", "cancer-LR-X.txt")
file_y_path = os.path.join(project_dir, "Dataset", "cancer-LR-y.txt")

X = None
y = None
try:
    X, y = load_cancer_dataset_matrices_for_LS(file_X_name=file_X_path, file_y_name=file_y_path)
except:
    main_generate_cancer_matrices_for_LS(file_X_name=file_X_path, file_y_name=file_y_path)
    X, y = load_cancer_dataset_matrices_for_LS()

In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

B, b = data_normalize_by_features(X, y)
A = concatenate_B_b(B, b)

In [4]:
def generate_default_configuration():
    kwargs = {
        'database': A,
        'r' : 300,
        'CI' : 0.95,
        'bootstrap_samples': 100, 
        'l2' : compute_largest_l2(A)
    }
    
    return kwargs

kwargs = generate_default_configuration()

In [5]:
mech = OptimalLS_mech(kwargs)

epsilon = 1
delta = 1/mech.n

mech.find_minimal_sigma(epsilon, delta)

7.917416411545128

In [6]:
mech = OptimalLS_mech(kwargs)

epsilon = 1
delta = 1/mech.n
num_samples = 100

tic = time.perf_counter()
samples = mech._gen_samples(epsilon, delta, num_samples)
toc = time.perf_counter()
logging.critical(f"cost {toc - tic:0.4f} seconds")

2024-12-09 13:49:30,048 cost 2.9121 seconds


In [7]:
tic = time.perf_counter()
accuracy_estimator = LS_accuracy_estimator(kwargs)
boot_res = accuracy_estimator.estimate_square_error(samples)
toc = time.perf_counter()
logging.critical(boot_res)
logging.critical(f"cost {toc - tic:0.4f} seconds")

2024-12-09 13:49:30,078 (bootstrap_result(lb_one_sided=0.9934134454261916, lb_two_sided=0.9908457274144706, ub_one_sided=1.015283658395076, ub_two_sided=1.0172217926678684), bootstrap_result(lb_one_sided=4.655879996185734, lb_two_sided=4.611210688927406, ub_one_sided=4.954623840424202, ub_two_sided=4.979501546586674))
2024-12-09 13:49:30,079 cost 0.0217 seconds


In [8]:
mech = LS_fromoptim_RP_mech(kwargs)

epsilon = 1
delta = 1/mech.n

mech.find_minimal_sigma(epsilon, delta)

/home/wei402/Desktop/NDIS/src/RP_mechanisms/optim_RP.py:25: RuntimeWarning: overflow encountered in exp
  exp_term = np.exp(epsilon - r / 2 * np.log(1 - leverage)) * (1 - leverage) ** (r / 2.0)
/home/wei402/Desktop/NDIS/src/RP_mechanisms/optim_RP.py:25: RuntimeWarning: invalid value encountered in double_scalars
  exp_term = np.exp(epsilon - r / 2 * np.log(1 - leverage)) * (1 - leverage) ** (r / 2.0)


6.850481817751286

In [9]:
mech = LS_fromoptim_RP_mech(kwargs)

epsilon = 1
delta = 1/mech.n
num_samples = 100

tic = time.perf_counter()
samples = mech._gen_samples(epsilon, delta, num_samples)
toc = time.perf_counter()
logging.critical(f"cost {toc - tic:0.4f} seconds")

2024-12-09 13:49:33,753 cost 3.6529 seconds


In [10]:
tic = time.perf_counter()
accuracy_estimator = LS_accuracy_estimator(kwargs)
boot_res = accuracy_estimator.estimate_square_error(samples)
toc = time.perf_counter()
logging.critical(boot_res)
logging.critical(f"cost {toc - tic:0.4f} seconds")

2024-12-09 13:49:33,779 (bootstrap_result(lb_one_sided=0.9885616200260687, lb_two_sided=0.9852351695906951, ub_one_sided=1.0116997806786534, ub_two_sided=1.0135634397193785), bootstrap_result(lb_one_sided=4.572130909755145, lb_two_sided=4.564900202687261, ub_one_sided=4.94343031022764, ub_two_sided=4.965550983013155))
2024-12-09 13:49:33,780 cost 0.0212 seconds


In [11]:
mech = ALT19LS_mech(kwargs)

epsilon = 1
delta = 1/mech.n

mech.compute_constant(epsilon, delta)

20.317246800063643

In [12]:
mech = ALT19LS_mech(kwargs)

epsilon = 1
delta = 1/mech.n
num_samples = 100

tic = time.perf_counter()
samples = mech._gen_samples(epsilon, delta, num_samples)
toc = time.perf_counter()
logging.critical(f"cost {toc - tic:0.4f} seconds")

2024-12-09 13:49:37,501 cost 3.6778 seconds


In [13]:
tic = time.perf_counter()
accuracy_estimator = LS_accuracy_estimator(kwargs)
boot_res = accuracy_estimator.estimate_square_error(samples)
toc = time.perf_counter()
logging.critical(boot_res)
logging.critical(f"cost {toc - tic:0.4f} seconds")

2024-12-09 13:49:37,530 (bootstrap_result(lb_one_sided=1.0086856522426992, lb_two_sided=1.007492163244649, ub_one_sided=1.0262400187021694, ub_two_sided=1.0273557378827438), bootstrap_result(lb_one_sided=4.8361316396624145, lb_two_sided=4.780837145158732, ub_one_sided=5.185719865788297, ub_two_sided=5.196437815072298))
2024-12-09 13:49:37,531 cost 0.0221 seconds
